In [35]:
#import packages
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
import math
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil import parser
import requests
from bs4 import BeautifulSoup as Soup
from datetime import date, timedelta
import datetime 
import dateutil.relativedelta
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [47]:
#load files
#game data
df_1617_gt = pd.read_csv('nba1617_gt.csv')  
df_1718_gt = pd.read_csv('nba1718_gt.csv')
frames = [df_1617_gt,df_1718_gt]
df_all_games = pd.concat(frames)

#viewership data
df_ILP = pd.read_csv('International_League_Pass_Viewership.csv')

#social data
df_gt = pd.read_csv('NBA_gts_161718.csv')
df_wp = pd.read_csv('nba_wp_pageviews.csv')
df_social = pd.read_csv('New_Social_Media_Metrics.csv')

In [48]:
#wikipedia page views
#convert to datetime
df_wp['date'] = pd.to_datetime(df_wp.date)

#extract month and year
df_wp['month'] = df_wp['date'].dt.month
df_wp['year'] = df_wp['date'].dt.year

#sum wikipedia pageviews
df_wp = df_wp.groupby(['team', 'month', 'year'])['wp_pageviews'].sum().reset_index()

In [49]:
#google trends

#sort date ascending
df_gt = df_gt.sort_values('date', ascending = True)

#convert to datetime
df_gt['date'] = pd.to_datetime(df_gt.date)

#extract month and year
df_gt['month'] = df_gt['date'].dt.month
df_gt['year'] = df_gt['date'].dt.year

#average of google trends scores in that month
df_gt = df_gt.groupby(['team', 'month', 'year'])['gts'].mean().reset_index()



In [50]:
#sum web metric score over a period of one month

#load team web data
df_web = pd.read_csv('Updated_Team_Web_Metrics.csv')

#"UNQ" = Total unique viewers of team website properties on the given Date; 
#"TTS" = Total time spent on team website properties on the given Date, in second
df_web['DTime'] = pd.to_datetime(df_web.Date) #converting to date to datetime
df_web['Date'] = df_web['DTime'].dt.date #converting to date to datetime

df_web = df_web.sort_values('Date',ascending=True)

#UNQ and TTS
df_web_unq = df_web[df_web['Metric']=='UNQ']
df_web_tts = df_web[df_web['Metric']=='TTS']

#make unq and tts columns
df_web_unq=df_web_unq.rename(columns = {'Value':'UNQ'})
df_web_tts=df_web_tts.rename(columns = {'Value':'TTS'})

#updated to contain columns for UNQ and TTS separately
df_web_updated = pd.merge(df_web_tts[['Team','Date','DTime','TTS']],df_web_unq[['Team','Date','UNQ']],on=['Date','Team'],how='left')


#filter web data for 2016-17 
df_web_updated = df_web_updated[df_web_updated.DTime >= '2016-05-01' ] 



#convert to date time
df_web_updated['date'] = pd.to_datetime(df_web_updated.Date)  

#extract month and year
df_web_updated['month'] = df_web_updated['date'].dt.month
df_web_updated['year'] = df_web_updated['date'].dt.year

print(len(df_web_updated))
#sum web metric pageviews
df_web_TTS = df_web_updated.groupby(['Team', 'month', 'year'])['TTS'].sum().reset_index()
df_web_UNQ = df_web_updated.groupby(['Team', 'month', 'year'])['UNQ'].mean().reset_index()


#
df_web_updated = pd.merge(df_web_TTS, df_web_UNQ, on=['Team', 'month', 'year'], how='left')




24780


In [51]:
#team abbreviation -> team name
team_abbrev_dict = {'New York Knicks':'NYK', 
                    'Utah Jazz':'UTA', 
                    'San Antonio Spurs':'SAS', 
                    'Miami Heat':'MIA', 
                    'Dallas Mavericks':'DAL', 
                    'Brooklyn Nets':'BRK', 
                    'Detroit Pistons':'DET', 
                    'Charlotte Hornets':'CHA', 
                    'Minnesota Timberwolves':'MIN',
                    'Oklahoma City Thunder':'OKC', 
                    'Sacramento Kings':'SAC', 
                    'Houston Rockets':'HOU', 
                    'Washington Wizards':'WAS', 
                    'Boston Celtics':'BOS', 
                    'Los Angeles Clippers':'LAC', 
                    'Cleveland Cavaliers':'CLE', 
                    'Indiana Pacers':'IND', 
                    'Orlando Magic':'ORL',
                    'Phoenix Suns':'PHX', 
                    'Los Angeles Lakers':'LAL', 
                    'Golden State Warriors':'GSW', 
                    'Memphis Grizzlies':'MEM', 
                    'New Orleans Pelicans':'NOP', 
                    'Portland Trail Blazers':'POR', 
                    'Milwaukee Bucks':'MIL', 
                    'Denver Nuggets':'DEN', 
                    'Chicago Bulls':'CHI',
                    'Toronto Raptors':'TOR', 
                    'Philadelphia 76ers':'PHI', 
                    'Atlanta Hawks':'ATL'
}

def map_abbr_team(team):
    """Maps team abbreviation to actual name"""
    team = team_abbrev_dict[team]
    return team




In [52]:
#map teams to team names and locations
df_all_games['Away'] = df_all_games['Away'].apply(lambda x:map_abbr_team(x))
df_all_games['Home'] = df_all_games['Home'].apply(lambda x:map_abbr_team(x))
df_gt['team'] = df_gt['team'].apply(lambda x:map_abbr_team(x))
df_wp['team'] = df_wp['team'].apply(lambda x:map_abbr_team(x))


In [53]:


#finds previous month and extract year for game data
df_all_games['Date'] = pd.to_datetime(df_all_games['Date'])
dateutil.relativedelta.relativedelta(months=-1)
df_all_games['prev_month'] = (df_all_games['Date'] - pd.DateOffset(months=1)).dt.month #previous month
df_all_games['year'] = (df_all_games['Date'] - pd.DateOffset(months=1)).dt.year #year of previous month
df_all_games['prev_month_2'] = (df_all_games['Date'] - pd.DateOffset(months=2)).dt.month #2 months ago
df_all_games['year_2'] = (df_all_games['Date'] - pd.DateOffset(months=2)).dt.year #year of 2 months ago
df_all_games['prev_month_3'] = (df_all_games['Date'] - pd.DateOffset(months=3)).dt.month #3 months ago
df_all_games['year_3'] = (df_all_games['Date'] - pd.DateOffset(months=3)).dt.year #year of 3 months ago
df_all_games['prev_month_4'] = (df_all_games['Date'] - pd.DateOffset(months=4)).dt.month #4 months ago
df_all_games['year_4'] = (df_all_games['Date'] - pd.DateOffset(months=4)).dt.year #4 months ago

In [54]:
#change names for merging purposes
df_social = df_social.rename(columns = {'Team':'team'})
df_web_updated = df_web_updated.rename(columns = {'Team':'team'})
df_social = df_social.rename(columns = {'Month':'month'})
df_social = df_social.rename(columns = {'Year':'year'})

#fix bug in data
df_social.loc[df_social.year == 720, 'year'] = 2018

In [55]:
#join social data
df_web_updated.head(3)
df_social = pd.merge(df_social, df_gt, on = ['team', 'month', 'year'], how='left')
df_social = pd.merge(df_social, df_wp, on = ['team', 'month', 'year'], how='left')
df_social = pd.merge(df_social, df_web_updated, on = ['team', 'month', 'year'], how='left')

In [56]:
#export data to csv
df_social.to_csv('social_nba.csv', index=False)